# Dataset

In this demo we will be using a dataset that contains social networking, tagging, and music artist listening information 
from a set of 2K users from **[Last.fm](http://www.last.fm)** online music system. 

**[Last.fm Dataset](https://grouplens.org/datasets/hetrec-2011/):** 

* 1892 users; 
* 17632 artists; 
* 12717 bi-directional user friend relations, i.e. 25434 (user_i, user_j) pairs; 
* 92834 user-listened artist relations, i.e. tuples [user, artist, listeningCount]; 
* 11946 tags; 186479 tag assignments (tas), i.e. tuples [user, tag, artist]. 
* Last Update May, 2011.
         

## Understanding the dataset

Access http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-readme.txt in order to know better the dataset

## Downloading the dataset:

You can use the link http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip

In [ ]:
!wget http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip -q --show-progress
!unzip hetrec2011-lastfm-2k.zip
!rm hetrec2011-lastfm-2k.zip

%env DATASET_PATH hetrec2011-lastfm-2k

# Import the dataset to Hive

In [ ]:
!beeline -u "jdbc:hive2://" -e "SHOW DATABASES;"

In [ ]:
!beeline -u "jdbc:hive2://" -e "USE musics; drop table user_artists;"

In [ ]:
#Creating table user_artists
!beeline -u "jdbc:hive2://" -e "USE musics; CREATE TABLE user_artists(                                        \
                                                                     userID   INT,                            \
                                                                     artistID INT,                            \
                                                                     weight   INT )                           \
                                                            ROW FORMAT DELIMITED                              \
                                                            FIELDS TERMINATED BY '\t'                         \
                                                            STORED AS TEXTFILE                                \
                                                            tblproperties(\"skip.header.line.count\"=\"1\");"

#Loading data
!beeline -u "jdbc:hive2://" -e "USE musics;  LOAD DATA LOCAL INPATH '$(pwd)/user_artists.dat'\
                                                OVERWRITE INTO TABLE user_artists;"

In [ ]:
!beeline -u "jdbc:hive2://" -e "USE musics; SELECT * FROM user_artists limit 10;"

In [ ]:
#Creating table artists
!beeline -u "jdbc:hive2://" -e "USE musics; CREATE TABLE artists(                                             \
                                                                     ID   INT,                                \
                                                                     name STRING,                             \
                                                                     URL   STRING,                             \
                                                                     pictureURL   STRING )                    \
                                                            ROW FORMAT DELIMITED                              \
                                                            FIELDS TERMINATED BY '\t'                         \
                                                            STORED AS TEXTFILE                                \
                                                            tblproperties(\"skip.header.line.count\"=\"1\");"

#Loading data
!beeline -u "jdbc:hive2://" -e "USE musics;  LOAD DATA LOCAL INPATH '$(pwd)/artists.dat'\
                                                OVERWRITE INTO TABLE artists;"


In [ ]:
!beeline -u "jdbc:hive2://" -e "USE musics; SELECT * FROM artists limit 10;"

In [ ]:
#Creating table user
!beeline -u "jdbc:hive2://" -e "USE musics; CREATE TABLE user_friends(                                       \
                                                                     userID    INT,                          \
                                                                     friendID  INT        )                  \
                                                            ROW FORMAT DELIMITED                             \
                                                            FIELDS TERMINATED BY '\t'                        \
                                                            STORED AS TEXTFILE                               \
                                                            tblproperties(\"skip.header.line.count\"=\"1\");"

#Loading data
!beeline -u "jdbc:hive2://" -e "USE musics;  LOAD DATA LOCAL INPATH '$(pwd)/user_friends.dat'\
                                                OVERWRITE INTO TABLE user_friends;"


In [ ]:
!beeline -u "jdbc:hive2://" -e "USE musics; SELECT * FROM user_friends limit 10;"

 # Exploring the dataset

## What is the name of the most popular artist (listened by distinct users)?

In [ ]:
!beeline -u "jdbc:hive2://" -e "USE musics; SELECT a.name, count(distinct b.userid) qd_user FROM artists a inner join user_artists b on b.artistid = a.id group by a.name order by qd_user desc limit 1;"

## What is the most popular user?

In [77]:
!beeline -u "jdbc:hive2://" -e "USE musics; \
                                SELECT a.userid  \
                                , count(distinct a.friendid) qd \
                                FROM user_friends a \
                                inner join user_artists b on b.userid = a.userid \
                                group by a.userid \
                                order by qd desc \
                                limit 1;"


Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.217 seconds)
Query ID = jovyan_20200930011254_568fdf19-8920-492a-afb9-0bcb1808e1de
Total jobs = 2
2020-09-30 01:13:01	Starting to launch local task to process map join;	maximum memory = 477626368
2020-09-30 01:13:01	Starting to launch local task to process map join;	maximum memory = 477626368
2020-09-30 01:13:03	Dump the side-table for tag: 0 with group count: 1892 into file: file:/tmp/jovyan/9b37f533-c5c2-4267-ba73-44be774d5bd2/hive_2020-09-30_01-12-54_607_610559708521159070-1/-local-10006/HashTable-Stage-2/MapJoin-mapfile00--.hashtable
2020-09-30 01:13:03	Uploaded 1 File to: file:/tmp/jovyan/9b37f533-c5c2-4267-ba73-44be774d5bd2/hive_2020-09-30_01-12-54_607_610559708521159070-1/-local-10006/HashTable-Stage-2/MapJoin-mapfile00--.hashtable (229686 bytes)
2020-09-30 01:13:03	End of local task; Time Taken: 1.725 s

## Which user most listened the band 'Enigma'?

In [79]:
!beeline -u "jdbc:hive2://" -e "USE musics; \
                                SELECT b.userid, max(b.weight) qd_user \
                                FROM artists a \
                                inner join user_artists b on b.artistid = a.id\
                                where a.name = 'Enigma' \
                                group by b.userid \
                                order by qd_user desc \
                                limit 1;"

Connecting to jdbc:hive2://
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
OK
No rows affected (1.275 seconds)
Query ID = jovyan_20200930011443_35d65b45-1f7f-41e9-af01-d7de2a3e6bd4
Total jobs = 2
2020-09-30 01:14:50	Starting to launch local task to process map join;	maximum memory = 477626368
2020-09-30 01:14:50	Starting to launch local task to process map join;	maximum memory = 477626368
2020-09-30 01:14:51	Dump the side-table for tag: 1 with group count: 17632 into file: file:/tmp/jovyan/b47368aa-fc69-43aa-b399-eee35d5aafb4/hive_2020-09-30_01-14-43_531_4140812516648328408-1/-local-10006/HashTable-Stage-2/MapJoin-mapfile01--.hashtable
2020-09-30 01:14:52	Uploaded 1 File to: file:/tmp/jovyan/b47368aa-fc69-43aa-b399-eee35d5aafb4/hive_2020-09-30_01-14-43_531_4140812516648328408-1/-local-10006/HashTable-Stage-2/MapJoin-mapfile01--.hashtable (1218228 bytes)
2020-09-30 01:14:52	End of local task; Time Taken: 2.0

# Challenge:

## What is the name of the most listened artist?

In [ ]:
!beeline -u "jdbc:hive2://" -e "USE musics; SELECT a.name, sum(b.weight) qd FROM artists a inner join user_artists b on b.artistid = a.id group by a.name order by qd desc limit 1;"